<a href="https://colab.research.google.com/github/IvanSerem/ds2-proyecto-agricola/blob/main/DataScienceII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

“¿Cómo se relacionan las condiciones climáticas y las propiedades del suelo en un punto geográfico determinado?”

“Para enriquecer el modelo predictivo desarrollado en Data Science I, se incorporaron variables climáticas históricas obtenidas desde la NASA POWER API, una fuente científica abierta diseñada para aplicaciones agrícolas y ambientales.”

“Las variables climáticas externas se extrajeron para el mismo período temporal que el dataset base, garantizando coherencia temporal entre las fuentes.”

“Las variables climáticas fueron agregadas temporalmente para representar condiciones promedio/acumuladas relevantes para el crecimiento del cultivo.”

1.1. Importación de Librerías y Definición de Parámetros

“Este proyecto integra el modelo analítico desarrollado en Data Science I con datos climáticos y edáficos obtenidos mediante APIs externas, permitiendo analizar las condiciones reales que influyen en el rendimiento agrícola y sentando las bases para modelos predictivos futuros.”

“El proyecto de Data Science II extiende el modelo conceptual desarrollado en Data Science I, incorporando datos climáticos y edáficos reales obtenidos mediante APIs externas, demostrando la continuidad metodológica entre ambas etapas.”

In [ ]:
# Importamos las librerías esenciales
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

# Configuramos un estilo visual para todos nuestros gráficos
sns.set_theme(style="whitegrid")

# Definimos el endpoint base de SoilGrids (ISRIC)
url_base = "https://rest.isric.org/soilgrids/v2.0/properties/query"

1.2. Extracción de Datos en Lotes

1.3. Inspeccionando los Resultados del Bucle

1.4. Consolidación y Verificación Finalmente, unimos todos los DataFrames de nuestra lista en una única tabla maestra con pd.concat y realizamos una primera verificación.